In [ ]:
# imports
import pandas as pd
import pickle
import networkx as nx
from vinal.build import grid_instance, distance_matrix
from concorde.tsp import TSPSolver # requires pyconcorde and mac
from InvokeLKH import * # requires LKH solver
import concorde

In [ ]:
# Multiply by 1000 because EUC_2D rounds to nearest integer
grid6x6 = grid_instance(6,6,manhattan=False)
df = pd.DataFrame()
df['x'] = nx.get_node_attributes(grid6x6,'x').values()
df['y'] = nx.get_node_attributes(grid6x6,'y').values()
grid6x6 = df*1000
us_cities_23 = pd.read_csv('data/us_cities_23.csv', index_col=0)*1000
xqf131 = pd.read_csv('data/xqf131.csv', index_col=0)*1000
pcb442 = pd.read_csv('data/pcb442.csv', index_col=0)*1000

In [ ]:
solver = TSPSolver.from_data(list(grid6x6.x), list(grid6x6.y), norm='EUC_2D')
grid6x6_tour = list(solver.solve().tour)
grid6x6_tour.append(grid6x6_tour[0])

solver = TSPSolver.from_data(list(us_cities_23.x), list(us_cities_23.y), norm='EUC_2D')
us_cities_23_tour = list(solver.solve().tour)
us_cities_23_tour.append(us_cities_23_tour[0])

solver = TSPSolver.from_data(list(xqf131.x), list(xqf131.y), norm='EUC_2D')
xqf131_tour = list(solver.solve().tour)
xqf131_tour.append(xqf131_tour[0])

solver = TSPSolver.from_data(list(pcb442.x), list(pcb442.y), norm='EUC_2D')
pcb442_tour = list(solver.solve().tour)
pcb442_tour.append(pcb442_tour[0])

In [ ]:
# use LKH to solve asymmetric TSP problem
xqf131_etching = pd.read_csv('data/xqf131_etching.csv', index_col=0)*1000
xqf131_etching_dm = distance_matrix(xqf131_etching, False, 'x_start', 'y_start', 'x_end', 'y_end').astype(int)
writeTSPLIBfile_FE('xqf131_etching', xqf131_etching_dm, 'An ATSP instance for solving a PCB etching problem.')
run_LKHsolver_cmd('xqf131_etching')
xqf131_etching_tour = [int(i[:-1])-1 for i in open('xqf131_etching.txt').readlines()[6:-2]]
xqf131_etching_tour.append(xqf131_etching_tour[0])
rm_solution_file_cmd('xqf131_etching')

In [ ]:
# remove pyconcorde generated files
os.system('rm Oname.res')
os.system('rm name.res')
os.system('rm name.sol')

In [ ]:
# create dictionary of optimal tours
optimal_tours = {}
optimal_tours['6x6_grid'] = grid6x6_tour
optimal_tours['us_cities_23'] = us_cities_23_tour
optimal_tours['xqf131'] = xqf131_tour
optimal_tours['pcb442'] = pcb442_tour
optimal_tours['xqf131_etching'] = xqf131_etching_tour

In [ ]:
with open('data/optimal_tours.pickle', 'wb') as f:
    pickle.dump(optimal_tours, f)